Importiamo Pyomo e GLPK

In [ ]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("glpk") or os.path.isfile("glpk")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk 
        except:
            pass

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 4.1 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 128275 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-2_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-2) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack ...

vogliamo modellare la scelta del mix produttivo ottimo dati un insieme di prodotti $P$ e un insieme di materie prime $M$. Per ogni prodottto conosciamo il profitto unitario, per ogni materia prima conosciamo la disponibilità. Inoltre conosciamo il consumo di ogni materia prima $m \in M$ per ottenere un'unità di prodotto $p \in P$ (la chiameremo $USO$)

Modello ILP:

$\max \sum_{p \in P} PROF_p x_p$

$\sum_{p \in P} USO_{m,p} x_p \le DISP_m m \in M$

$x_p ~intere ~~~~p \in P$

$PROF$=profitto

In [ ]:
M = ['acciaio', 'legno', 'plastica', 'gomma', 'vetro', 'vernice'] #materia prima
P = [1,2,3,4,5,6] #prodotti
#USO = {...} 
#DISP = {...}
#PROF = {1:,2 ....} 
#Anziche scrivere questi dizionari e matrici a mano, possiamo automatizzarne la scrittura

In [ ]:
u = [1,4,0,4,2,0,
     4,5,3,0,1,0,
     0,3,8,0,1,0,
     2,0,1,2,1,5,
     2,4,2,2,2,4,
     1,4,1,4,3,4] #sono le words che vanno nel nostro dizionario

In [ ]:
keys = [] #creo una lista vuota
for m in M: #per ongi materia nell'insieme delle materie
  for p in P: #per ogni p nell'insieme dei prodotti
    keys = keys + [(m,p)] #alla lista delle chiavi aggiunge un altro valore della lista
print(keys)
USO = dict(zip(keys,u)) #dizionario
print(USO)

[('acciaio', 1), ('acciaio', 2), ('acciaio', 3), ('acciaio', 4), ('acciaio', 5), ('acciaio', 6), ('legno', 1), ('legno', 2), ('legno', 3), ('legno', 4), ('legno', 5), ('legno', 6), ('plastica', 1), ('plastica', 2), ('plastica', 3), ('plastica', 4), ('plastica', 5), ('plastica', 6), ('gomma', 1), ('gomma', 2), ('gomma', 3), ('gomma', 4), ('gomma', 5), ('gomma', 6), ('vetro', 1), ('vetro', 2), ('vetro', 3), ('vetro', 4), ('vetro', 5), ('vetro', 6), ('vernice', 1), ('vernice', 2), ('vernice', 3), ('vernice', 4), ('vernice', 5), ('vernice', 6)]
{('acciaio', 1): 1, ('acciaio', 2): 4, ('acciaio', 3): 0, ('acciaio', 4): 4, ('acciaio', 5): 2, ('acciaio', 6): 0, ('legno', 1): 4, ('legno', 2): 5, ('legno', 3): 3, ('legno', 4): 0, ('legno', 5): 1, ('legno', 6): 0, ('plastica', 1): 0, ('plastica', 2): 3, ('plastica', 3): 8, ('plastica', 4): 0, ('plastica', 5): 1, ('plastica', 6): 0, ('gomma', 1): 2, ('gomma', 2): 0, ('gomma', 3): 1, ('gomma', 4): 2, ('gomma', 5): 1, ('gomma', 6): 5, ('vetro', 1): 

In [ ]:
DISP = dict(zip(M, [800, 1160, 1780, 1050, 1360, 1240]))
print(DISP)
PROF = dict(zip(P,[30, 45, 24, 26, 24, 30]))
print(PROF)

{'acciaio': 800, 'legno': 1160, 'plastica': 1780, 'gomma': 1050, 'vetro': 1360, 'vernice': 1240}
{1: 30, 2: 45, 3: 24, 4: 26, 5: 24, 6: 30}


Importiamo la libreria PYOMO

In [ ]:
from pyomo.environ import * #qui andrebbe a seguire un nome che diventi quella che usiamo nel codice, best practice


In [ ]:
mix = ConcreteModel() #chiamo il modello mix

Definiamo delle variabili non negative

In [ ]:
mix.x = Var(P, within=NonNegativeIntegers)

Definiamo la funzione obiettivo ma per farlo definiamo una funzione che prende come argomento il modello(chiamato genericamente mod, parametro informale) che poi passiamo alla funzione come rule

In [ ]:
def obj_rule(mod):
  return sum(PROF[p]*mod.x[p] for p in P)
mix.obj = Objective(rule=obj_rule, sense = maximize) #la funzione obiettivo si aspetta o una espressione o una regola per 
#nella sintassi usata ieri col KP01: 
#Objective(expr=sum(PROF[p]*mix.x[p] for p in P))
#invece oggi creiamo uan funzione 

Allo stesso modo definiamo i vincoli. Innanzitutto definiamo una funzione che prende come argomento il modello (mod) e un parametro m.

Attenzione: abbiamo un vincolo per ogni materiale, cioe per ogni elemento $m$ della lista $M$

In [ ]:
def disponibilita_rule(mod, m):
  return sum(USO[m, p] * mod.x[p] for p in P) <= DISP[m]
mix.disponibilita_const = Constraint(M, rule=disponibilita_rule) #qui pyomo prende ogni elemento dell'insieme M, chiama la funzione

In [ ]:
SolverFactory('glpk').solve(mix, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmpjwhiuruu.glpk.raw --wglp /tmp/tmpmquh1pm3.glpk.glp --cpxlp
 /tmp/tmphuq1rr7a.pyomo.lp
Reading problem data from '/tmp/tmphuq1rr7a.pyomo.lp'...
7 rows, 7 columns, 29 non-zeros
6 integer variables, none of which are binary
77 lines were read
Writing problem data to '/tmp/tmpmquh1pm3.glpk.glp'...
66 lines were written
GLPK Integer Optimizer, v4.65
7 rows, 7 columns, 29 non-zeros
6 integer variables, none of which are binary
Preprocessing...
6 rows, 6 columns, 28 non-zeros
6 integer variables, none of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  8.000e+00  ratio =  8.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 6
Solving LP relaxation...
GLPK Simplex Optimizer, v4.65
6 rows, 6 columns, 28 non-zeros
*     0: obj =  -0.000000000e+00 inf =   0.000e+00 (6)
*     9: obj =   1.502000000e+04 inf =   0.000e+00 (0)
OPTIMAL LP SO

{'Problem': [{'Name': 'unknown', 'Lower bound': 15020.0, 'Upper bound': 15020.0, 'Number of objectives': 1, 'Number of constraints': 7, 'Number of variables': 7, 'Number of nonzeros': 29, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.031273841857910156}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [ ]:
mix.pprint()

2 Set Declarations
    disponibilita_const_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {'acciaio', 'legno', 'plastica', 'gomma', 'vetro', 'vernice'}
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {1, 2, 3, 4, 5, 6}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 : 120.0 :  None : False : False : NonNegativeIntegers
          2 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
          3 :     0 : 220.0 :  None : False : False : NonNegativeIntegers
          4 :     0 : 160.0 :  None : False : False : NonNegativeIntegers
          5 :     0 :  20.0 :  None : False : False : NonNegativeIntegers
          6 :     0 :  50.0 :  None : False : False : NonNegativeIntegers

1 Objective Declarations
    obj : Size=1, Index=N